In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from system import *

In [2]:
from tqdm.notebook import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings

warnings.filterwarnings("ignore")
tf.keras.backend.set_floatx("float64")

%load_ext autoreload
%autoreload 2

In [6]:
class Ansatz(tf.keras.Model):
    def __init__(self, nodes, dim):
        super().__init__()
        self.dim = dim
        
        self.Layers = []
        for node in nodes:
            self.Layers.append(tf.keras.layers.Dense(node))
    
        self.out = tf.keras.layers.Dense(1)

    def call(self, x):
        for layer in self.Layers:
            x = tf.math.tanh(layer(x))

        return tf.math.exp(self.out(x))   
    
  
@tf.function
def loss(system, x):
    batch_size = x.shape[1]
    _x = tf.unstack(x, axis=1)
    _x_ = [tf.expand_dims(tmp, axis = 1) for tmp in _x]
    _x2 = tf.transpose(tf.stack(_x_))[0]


    psi = system.Ansatz(_x2)
    d_psi = [tf.squeeze(tf.gradients(psi, tmp)) for tmp in _x]
    d2_psi = 0
    for ii in range(batch_size):
        d2_psi += tf.stack(tf.gradients(d_psi[ii], _x_[ii]))[0]
    
    lap = 1/psi*(d2_psi)
    
    E_L = -0.5*lap + 0.5*tf.reshape(tf.reduce_sum(x**2, axis=1), (-1,1)) 
    E_L = tf.stop_gradient(E_L)
    E = tf.math.reduce_mean(E_L)
    
    psi_detach = tf.stop_gradient(psi)
    return E, 2*(tf.math.reduce_mean(psi/psi_detach*E_L) - tf.math.reduce_mean(psi/psi_detach)*E)


@tf.function
def grad(system, x):
    with tf.GradientTape() as tape:
        E, loss_value = loss(system, x)

    return E, tape.gradient(loss_value, system.Ansatz.trainable_variables)


def train(system, num_epochs, batch_size, optimizer, verbose = True):
    energy = []
    
    for epoch in tqdm(range(num_epochs)):
        x = system.Sampler(batch_size)[0]

        E, gradients = grad(system, x)
        energy.append(E)

        optimizer.apply_gradients(zip(gradients, system.Ansatz.trainable_variables))
        
        if verbose:
            if epoch%100 == 0: 
                print(f"epoch: {epoch}, Energy: {E}")
    
    return energy  

In [7]:
tf.random.set_seed(42)
np.random.seed(42)

nodes = ((32, 32))
dim = 3

ansatz = Ansatz(nodes, dim)
external = HarmonicOsc(0.5)
sampler = Metropolis(1, 20)

system = System(num_part = 1,
                 dim = 3,
                 Ansatz=ansatz,
                 External=external,
                 Internal=None,
                 Sampler=sampler
                 )

optimizer = tf.keras.optimizers.Adam(0.01)
energy = []

In [8]:
num_epochs = 1000
batch_size = 2000

energy = train(system, num_epochs, batch_size, optimizer, verbose = True)

epoch: 0, Energy: 11.076157628333785
epoch: 100, Energy: 1.531682920821983
epoch: 200, Energy: 1.504612264621102
epoch: 300, Energy: 1.5030059318629851
epoch: 400, Energy: 1.4994223920728469
epoch: 500, Energy: 1.4956759277921738
epoch: 600, Energy: 1.500158624441236
epoch: 700, Energy: 1.50261095535419
epoch: 800, Energy: 1.4992189424454545
epoch: 900, Energy: 1.4982225814851449

